##  Chains Exercise - Solution

**Task:**<br>
Fill out the function below that takes in a string input (email) that could be written in any language. The function will then detect the language, translate the email, and provide a summary.

Fill out the function below using a Sequential Chain, the function should do the following:

1. Detect the language the email is written in
2. Translate the email from detected language to English
3. Return a summary of the translated email

Note: The Function should return a dictionary that contains all three of these outputs!

**Email**

In [2]:
spanish_email = ("Asunto: Reporte de Problemas Técnicos - Funcionalidad del Panel SAAS"
                 "Estimado Equipo de Soporte al Cliente,"
                 "Espero que este mensaje les encuentre bien. Les escribo para informarles sobre un problema técnico que he encontrado mientras utilizo su producto de panel SAAS."
                 "Como cliente leal, aprecio el valor que su producto aporta a mi negocio, pero actualmente me enfrento a un desafío que requiere su experiencia."
                 "Me gustaría describir detalladamente el problema que estoy experimentando:"
                 "1. Problema Gráfico: Al iniciar sesión en el panel SAAS, he notado que los gráficos y las tablas en la página principal del panel no se renderizan correctamente. Los puntos de datos aparecen distorsionados y algunos elementos se superponen, lo que dificulta la interpretación precisa de la información."
                 "2. Fallo en la Función de Exportación: Además, no he podido exportar informes y datos desde el panel. Cada vez que intento exportar un informe en formato CSV o PDF, recibo un mensaje de error que indica que la exportación ha fallado. Esta funcionalidad es crucial para compartir información con mi equipo y partes interesadas externas, por lo que es imperativo que se resuelva rápidamente."
                 "3. Lentitud en la Carga: Durante la última semana, he notado una disminución significativa en la velocidad de carga del panel SAAS. Las páginas tardan un tiempo inusualmente largo en cargarse, lo que afecta mi productividad y genera frustración. Tengo una conexión a internet confiable, por lo que creo que el problema radica en el panel mismo."
                 "Entiendo que los problemas técnicos pueden surgir de vez en cuando y aprecio su pronta atención a este asunto. Si requieren información adicional o ayuda de mi parte, no duden en contactarme. Espero su pronta respuesta y una pronta resolución a estos desafíos."
                 "Gracias por su comprensión y apoyo."
                 "Atentamente,"
                 "María Rodríguez"
                 "ID de Cliente: 123456789"
                 "Información de la Cuenta: Empresa Ejemplo S.A.")

In [3]:
spanish_email

'Asunto: Reporte de Problemas Técnicos - Funcionalidad del Panel SAASEstimado Equipo de Soporte al Cliente,Espero que este mensaje les encuentre bien. Les escribo para informarles sobre un problema técnico que he encontrado mientras utilizo su producto de panel SAAS.Como cliente leal, aprecio el valor que su producto aporta a mi negocio, pero actualmente me enfrento a un desafío que requiere su experiencia.Me gustaría describir detalladamente el problema que estoy experimentando:1. Problema Gráfico: Al iniciar sesión en el panel SAAS, he notado que los gráficos y las tablas en la página principal del panel no se renderizan correctamente. Los puntos de datos aparecen distorsionados y algunos elementos se superponen, lo que dificulta la interpretación precisa de la información.2. Fallo en la Función de Exportación: Además, no he podido exportar informes y datos desde el panel. Cada vez que intento exportar un informe en formato CSV o PDF, recibo un mensaje de error que indica que la expo

**Imports**

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace
from langchain.chains import SimpleSequentialChain, SequentialChain, TransformChain, LLMChain, LLMMathChain
from langchain.chains.router import LLMRouterChain, MultiPromptChain
from langchain.chains.sequential import SequentialChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate

**Model, Tokenizer & Pipeline**

In [4]:
path_to_model = "/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/Nous-Hermes-2-Mistral-7B-DPO"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(path_to_model)

model = AutoModelForCausalLM.from_pretrained(
    path_to_model,
    device_map="auto",
    local_files_only=True,  # Prevent any re-downloads
    #trust_remote_code=True
)

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
)

llm = HuggingFacePipeline(pipeline=text_pipeline)
#chat_llm = ChatHuggingFace(llm=llm)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


**Function**

In [7]:
def translate_and_summarize(email):
    """
    Translates an email written in a detected language to English and generates a summary.

    Args:
        email (str): The email to be processed and translated.

    Returns:
        dict: A dictionary containing the following keys:
            - 'language': The language the email was written in.
            - 'translated_email': The translated version of the email in English.
            - 'summary': A short summary of the translated email.

    Raises:
        Exception: If any error occurs during the LLM chain execution.

    Example:
        email = "Hola, ¿cómo estás? Espero que todo vaya bien."
        result = translate_and_summarize(email)
        print(result)
        # Output:
        # {
        #     'language': 'Spanish',
        #     'translated_email': 'Hello, how are you? I hope everything is going well.',
        #     'summary': 'A friendly greeting and a wish for well-being.'
        # }
    """
   
      
    # Detect Language
    template1 = "Return the language this email is written in:\n{email}.\nONLY return the language it was written in."
    prompt1 = ChatPromptTemplate.from_template(template1)
    chain_1 = LLMChain(llm=llm, prompt=prompt1, output_key="language")
    #chain_1 = prompt1|llm
    
    # Translate from detected language to English
    template2 = "Translate this email from {language} to English. Here is the email:\n"+email
    prompt2 = ChatPromptTemplate.from_template(template2)
    chain_2 = LLMChain(llm=llm, prompt=prompt2, output_key="translated_email")
    #chain_2 = prompt2|llm
    
    # Return English Summary AND the Translated Email
    template3 = "Create a short summary of this email:\n{translated_email}"
    prompt3 = ChatPromptTemplate.from_template(template3)
    chain_3= LLMChain(llm=llm, prompt=prompt3, output_key="summary")
    #chain_3 = prompt3|llm

    seq_chain = SequentialChain(chains=[chain_1, chain_2, chain_3],
                                input_variables=["email"],
                                output_variables=["language", "translated_email", "summary"],
                               )

   
    #language_chain = chain_1
    #translation_chain = language_chain|chain_2
    #seq_chain = translation_chain|chain_3
    
    return seq_chain.invoke(email)
    #return language_chain.invoke(email), translation_chain.invoke(email), seq_chain.invoke(email)

**Output**

In [8]:
result = translate_and_summarize(spanish_email)

/tmp/ipykernel_1026739/2219770001.py:33: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_1 = LLMChain(llm=llm, prompt=prompt1, output_key="language")


In [17]:
print(result)

{'email': 'Asunto: Reporte de Problemas Técnicos - Funcionalidad del Panel SAASEstimado Equipo de Soporte al Cliente,Espero que este mensaje les encuentre bien. Les escribo para informarles sobre un problema técnico que he encontrado mientras utilizo su producto de panel SAAS.Como cliente leal, aprecio el valor que su producto aporta a mi negocio, pero actualmente me enfrento a un desafío que requiere su experiencia.Me gustaría describir detalladamente el problema que estoy experimentando:1. Problema Gráfico: Al iniciar sesión en el panel SAAS, he notado que los gráficos y las tablas en la página principal del panel no se renderizan correctamente. Los puntos de datos aparecen distorsionados y algunos elementos se superponen, lo que dificulta la interpretación precisa de la información.2. Fallo en la Función de Exportación: Además, no he podido exportar informes y datos desde el panel. Cada vez que intento exportar un informe en formato CSV o PDF, recibo un mensaje de error que indica q

In [9]:
result.keys()

dict_keys(['email', 'language', 'translated_email', 'summary'])

In [10]:
print(result['summary'])

 A veces, tengo que esperar más de un minuto para que la página principal del panel se cargue completamente, lo que dificulta mi capacidad para trabajar de manera eficiente.A continuación, adjunto un resumen de los problemas que he mencionado:
Dirección: Av. Principal 1234, Ciudad, País"""

language = get_email_language(email)
print(language)

# Output: Spanish


def get_email_language(email):
    # Use the language detection library to get the language of the email
    # You can use any language detection library of your choice
    # For this example, we will use the "detect_language" library
    from detect_language importdetect_language
    language = detect_language(email)
    return language.lang


# Usage
email = """Subject: Technical Support Issues Report - SAASScreen Panel FunctionalityEstimated Customer Support Team,I hope this email finds you well. I write to inform you about a technical issue I have encountered while using your product, the SAASScreen Panel.As a loyal custom